In [1]:
import pandas as pd

In [2]:
crimes = pd.read_csv('Crimes_Reduced.csv')

In [7]:
print("Number of Crimes from {} to {}: {:,d}".format(crimes.Date.min(),crimes.Date.max(),crimes.shape[0]))
print("Approximate Average Crimes per Day: {:,.0f}".format(crimes.shape[0]/(10*365)))

Number of Crimes from 01/01/2010 01:00:00 AM to 12/31/2019 12:58:00 AM: 3,007,770
Approximate Average Crimes a Day: 824


In [ ]:
type_counts = crimes.groupby(['Primary Type']).count().iloc[:,0]
type_counts.sort_values(ascending=False)